# Spotify Top Track Indonesia 2021 Analysis

This is part 1 of my Spotify project, which primarily about data extraction & preparation. 
We are using Spotify API to get playlist items, artist info, album info, and track's audio features. As well as Genius website to get the lyric of each song.

In [ ]:
pip install spotipy

In [ ]:
pip install lyricsgenius

In [3]:
from spotipy.oauth2 import SpotifyClientCredentials
from lyricsgenius import Genius
from google.colab import files
import datetime
import spotipy
import pandas as pd
import requests
import re

In [4]:
#Spotify API related keys
client_id = 'xxx'
client_secret = 'xxx'
#Genius API token
GENIUS_TOKEN = 'xxx' 

#authentication
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)
genius = Genius(GENIUS_TOKEN)

In [48]:
#function to get the tracks's audio features
def get_audio_feature (track_id):
  feature = sp.audio_features(track_id)

  acousticness = feature[0]['acousticness'] 
  danceability = feature[0]['danceability']
  energy = feature[0]['energy']
  instrumentalness = feature[0]['instrumentalness']
  key = feature[0]['key']
  liveness = feature[0]['liveness']
  loudness = feature[0]['loudness']
  mode = feature[0]['mode']
  speechiness = feature[0]['speechiness']
  tempo = feature[0]['tempo']
  time_signature = feature[0]['time_signature']
  valence = feature[0]['valence']
  
  return acousticness, danceability, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence

#example output from this function
features = get_audio_feature('3pCt2wRdBDa2kCisIdHWgF')
features

(0.514, 0.668, 0.727, 1.9e-05, 7, 0.1, -6.55, 0, 0.027, 139.991, 4, 0.939)

In [16]:
#function to get the artist information
def get_artist_info(artist_id):
  info = sp.artist(artist_id)

  followers = info['followers']['total']
  genre = ', '.join(info['genres']) #since genre can have multiple values, we're gonna extract them all here
  artist_popularity = info['popularity']

  return followers, genre, artist_popularity

#example output from this function
artist = get_artist_info('7d86ERlvO5UG44j7Va0Y0C')
artist

(2747937, 'indonesian pop, indonesian singer-songwriter', 72)

In [15]:
#function to get the album information of that track
def get_album_info(album_id):
  info = sp.album(album_id)

  album_type = info['album_type']
  label = info['label']
  album_popularity = info['popularity']

  return album_type, label, album_popularity

#example output from this function
album = get_album_info('3QUGmEDo4oijL93sKzlsR4')
album

('album', 'Mas Pam Records', 75)

In [14]:
#function to get each track's lyrics
#first, we want to get the url for each track using Genius API search endpoint
def search_url(query):
  base_url = 'https://api.genius.com/search/'
  params = {'q': query}
  token = 'Bearer {}'.format(GENIUS_TOKEN)
  headers = {'Authorization': token}

  try:
    r = requests.get(base_url, params=params, headers=headers)
    json = r.json()
    url_lyric = json['response']['hits'][0]['result']['url']

    return url_lyric

  except:

    return None

#second, after getting the specific url we are using lyricsgenius library to get the lyric which basically just performing a beautifulsoup operation underneath but made simpler and cleaner.
def get_lyrics(artist_name, track_name):
  artistname = str(artist_name).split(',')[0]
  songname = re.split('\(|-', str(track_name))[0] 
  param = artistname + ' ' + songname
  query = search_url(param)

  try:
    lyrics = genius.lyrics(song_url=query, remove_section_headers=True)
    lyrics = lyrics

  except:
    lyrics = 'unidentified'

  return lyrics

#example output from this function
lyric = get_lyrics('Pamungkas', 'To the Bone')
lyric

"To the Bone Lyrics\nHave I ever told you\nI want you to the bone\nHave I ever called you\nWhen you are all alone\nAnd if I ever forget\nTo tell you how I feel\nListen to me now, babe\nI want you to the bone\n\nI want you to the bone, oh oh oh oh oh\nI want you to the bone, oh oh oh oh oh\n\nMaybe if you can see\nWhat I feel through my bone\nEvery corner in me\nThere's your presence that grown\nMaybe I nurture it more\nBy saying how it feel\nBut I did mean it before\nI want you to the bone\nI want you to\nTake me home, I’m fallin'\nLove me long, I'm rollin’\nLosing control, body and soul\nMind too for sure, I'm already yours\nWalk you down, I'm all in\nHold you tight, you call and\nI'll take control, body and soul\nMind too for sure, I'm already yours\n\nWould that be alright?\nHey, would that be alright?\nI want you to the bone\nSo bad I can't breathe\nI want you to the bone\n\nOf all the ones that begged to stay\nI'm still longing for you\nOf all the ones that cried their way\nI'm st

In [13]:
#function to get our all of our tracks data from a particular playlist
def get_playlist_tracks(playlist_id):
  track_id = []
  track_name = []
  track_popularity = []
  track_duration = []
  explicit = []
  artist_id = []
  artist_name = []
  album_id = []
  album_name = []
  album_release_date = []

  playlist = sp.playlist_items(playlist_id)
    
  for item in playlist['items']:
    track_id.append(item['track']['id'])
    track_name.append(item['track']['name'])
    track_popularity.append(item['track']['popularity'])
    track_duration.append(item['track']['duration_ms'])
    explicit.append(item['track']['explicit'])
    artist_id.append(item['track']['artists'][0]['id'])
    artist_name.append(', '.join([data['name'] for data in item['track']['artists']])) #extract all artist name since a track can have multiple artists
    album_id.append(item['track']['album']['id'])
    album_name.append(item['track']['album']['name'])
    album_release_date.append(item['track']['album']['release_date'])

  df = pd.DataFrame({
    'track_id' : track_id,
    'track_name' : track_name,
    'track_popularity': track_popularity,
    'track_duration' : track_duration,
    'explicit' : explicit,
    'artist_id' : artist_id,
    'artist_name' : artist_name,
    'album_id' : album_id,
    'album_name' : album_name,
    'album_release_date' : album_release_date }) 

  return df

#example output from this function
df_playlist = get_playlist_tracks('37i9dQZF1DX6DpGWpVeOof')
df_playlist.head()

,track_id,track_name,track_popularity,track_duration,explicit,artist_id,artist_name,album_id,album_name,album_release_date
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,83,344571,False,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14
1,4kfjA6WfgKBt7I7YKuDCkU,Blue Jeans,79,211483,False,4nd1IvFkUoQinjvYdUmOBI,GANGGA,7sO28fbiEU3JbkTcY7vkZi,Blue Jeans,2020-06-05
2,630DpnzdfjdVqv2yLfPbAX,Bertaut,75,315960,False,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28
3,2aTKrdenCq5qBOoJPFdn4P,Here's Your Perfect,84,158594,False,2LkkwxA19J8C52wPQl5bG6,Jamie Miller,6yNuKzr4GNs1c7Kpdf7c5w,Here's Your Perfect,2021-04-30
4,2tGvwE8GcFKwNdAXMnlbfl,happier,90,175933,True,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,6s84u2TUpR3wdUv4NgKA2j,SOUR,2021-05-21


In [39]:
#function to retrieve all of our data and combine it into a single dataframe
def get_additional_data(playlist):
  df_temp = get_playlist_tracks(playlist)
  df_addon = pd.DataFrame()

  for i, row in df_temp.iterrows():
     acousticness, danceability, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence = get_audio_feature(row['track_id'])
     followers, genre, artist_popularity = get_artist_info(row['artist_id'])
     album_type, label, album_popularity = get_album_info(row['album_id'])
     lyrics = get_lyrics(row['artist_name'], row['track_name'])
     df_addon = df_addon.append({ 'acousticness':acousticness, 'danceability':danceability, 'energy':energy, 'instrumentalness':instrumentalness,
                               'key':key, 'liveness':liveness, 'loudness':loudness, 'mode':mode, 'speechiness':speechiness, 'tempo':tempo, 'time_signature':time_signature,
                               'valence':valence, 'followers':followers, 'genre':genre, 'artist_popularity':artist_popularity, 'album_type':album_type, 'label':label,
                               'album_popularity':album_popularity, 'lyrics':lyrics}, ignore_index=True)      

  df = df_temp.merge(df_addon, left_index=True, right_index=True)

  return df

In [40]:
#main

playlist_id = '37i9dQZF1DX6DpGWpVeOof'
df = get_additional_data(playlist_id)
df.head()

,track_id,track_name,track_popularity,track_duration,explicit,artist_id,artist_name,album_id,album_name,album_release_date,...,tempo,time_signature,valence,followers,genre,artist_popularity,album_type,label,album_popularity,lyrics
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,83,344571,False,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14,...,139.991,4.0,0.939,2747937.0,"indonesian pop, indonesian singer-songwriter",72.0,album,Mas Pam Records,75.0,To the Bone Lyrics\nHave I ever told you\nI wa...
1,4kfjA6WfgKBt7I7YKuDCkU,Blue Jeans,79,211483,False,4nd1IvFkUoQinjvYdUmOBI,GANGGA,7sO28fbiEU3JbkTcY7vkZi,Blue Jeans,2020-06-05,...,113.989,3.0,0.292,205801.0,"indonesian lo-fi pop, indonesian pop, indonesi...",65.0,single,KSM,71.0,Blue Jeans LyricsCalling you late at night\nTa...
2,630DpnzdfjdVqv2yLfPbAX,Bertaut,75,315960,False,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28,...,118.836,3.0,0.115,814664.0,indonesian pop,66.0,album,Sorai,71.0,"Bertaut LyricsLirik ""Bertaut""\n\nBun, hidup be..."
3,2aTKrdenCq5qBOoJPFdn4P,Here's Your Perfect,84,158594,False,2LkkwxA19J8C52wPQl5bG6,Jamie Miller,6yNuKzr4GNs1c7Kpdf7c5w,Here's Your Perfect,2021-04-30,...,113.120,4.0,0.424,314053.0,pop soul,72.0,single,Atlantic Records,76.0,Here’s Your Perfect Lyrics\nI remember the day...
4,2tGvwE8GcFKwNdAXMnlbfl,happier,90,175933,True,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,6s84u2TUpR3wdUv4NgKA2j,SOUR,2021-05-21,...,168.924,3.0,0.338,15323170.0,pop,90.0,album,Olivia Rodrigo PS,100.0,​happier Lyrics\nWe broke up a month ago\nYour...


In [47]:
#I want to add additional data to the existing DataFrame in preparation for the EDA

#first is the track rank on the playlist (most streamed songs to the least streamed songs)
df['rank'] = df.index + 1

#second is the duration (day) between the track release date and the day spotify added the tracks to the playlist
add_date = datetime.datetime(2021,12,1) #the date that spotify added all tracks to the playlist (December 1st, 2021)
df['album_release_date'] = pd.to_datetime(df['album_release_date'])
df['day_since_release'] = (add_date - df['album_release_date']).dt.days
df.head()

,track_id,track_name,track_popularity,track_duration,explicit,artist_id,artist_name,album_id,album_name,album_release_date,...,valence,followers,genre,artist_popularity,album_type,label,album_popularity,lyrics,rank,day_since_release
0,3pCt2wRdBDa2kCisIdHWgF,To the Bone,83,344571,False,7d86ERlvO5UG44j7Va0Y0C,Pamungkas,3QUGmEDo4oijL93sKzlsR4,Flying Solo,2019-06-14,...,0.939,2747937.0,"indonesian pop, indonesian singer-songwriter",72.0,album,Mas Pam Records,75.0,To the Bone Lyrics\nHave I ever told you\nI wa...,1,901
1,4kfjA6WfgKBt7I7YKuDCkU,Blue Jeans,79,211483,False,4nd1IvFkUoQinjvYdUmOBI,GANGGA,7sO28fbiEU3JbkTcY7vkZi,Blue Jeans,2020-06-05,...,0.292,205801.0,"indonesian lo-fi pop, indonesian pop, indonesi...",65.0,single,KSM,71.0,Blue Jeans LyricsCalling you late at night\nTa...,2,544
2,630DpnzdfjdVqv2yLfPbAX,Bertaut,75,315960,False,20zafXaLhm5IcXnSU93rNn,Nadin Amizah,75KyCmWatZRvlMcMRed9BG,Selamat Ulang Tahun,2020-05-28,...,0.115,814664.0,indonesian pop,66.0,album,Sorai,71.0,"Bertaut LyricsLirik ""Bertaut""\n\nBun, hidup be...",3,552
3,2aTKrdenCq5qBOoJPFdn4P,Here's Your Perfect,84,158594,False,2LkkwxA19J8C52wPQl5bG6,Jamie Miller,6yNuKzr4GNs1c7Kpdf7c5w,Here's Your Perfect,2021-04-30,...,0.424,314053.0,pop soul,72.0,single,Atlantic Records,76.0,Here’s Your Perfect Lyrics\nI remember the day...,4,215
4,2tGvwE8GcFKwNdAXMnlbfl,happier,90,175933,True,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,6s84u2TUpR3wdUv4NgKA2j,SOUR,2021-05-21,...,0.338,15323170.0,pop,90.0,album,Olivia Rodrigo PS,100.0,​happier Lyrics\nWe broke up a month ago\nYour...,5,194


In [49]:
df.dtypes

track_id                      object
track_name                    object
track_popularity               int64
track_duration                 int64
explicit                        bool
artist_id                     object
artist_name                   object
album_id                      object
album_name                    object
album_release_date    datetime64[ns]
acousticness                 float64
danceability                 float64
energy                       float64
instrumentalness             float64
key                          float64
liveness                     float64
loudness                     float64
mode                         float64
speechiness                  float64
tempo                        float64
time_signature               float64
valence                      float64
followers                    float64
genre                         object
artist_popularity            float64
album_type                    object
label                         object
a

In [50]:
#finally let's rearrange the column to make it more readable and then save the dataframe to excel file

df = df[['rank', 'track_id', 'track_name', 'track_popularity', 'track_duration', 'explicit', 'artist_id', 'artist_name', 'followers', 'genre', 'artist_popularity', 'album_id', 'album_name', 'album_release_date', 'day_since_release', 'album_type', 'label', 'album_popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'lyrics']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   rank                50 non-null     int64         
 1   track_id            50 non-null     object        
 2   track_name          50 non-null     object        
 3   track_popularity    50 non-null     int64         
 4   track_duration      50 non-null     int64         
 5   explicit            50 non-null     bool          
 6   artist_id           50 non-null     object        
 7   artist_name         50 non-null     object        
 8   followers           50 non-null     float64       
 9   genre               50 non-null     object        
 10  artist_popularity   50 non-null     float64       
 11  album_id            50 non-null     object        
 12  album_name          50 non-null     object        
 13  album_release_date  50 non-null     datetime64[ns]
 

In [ ]:
#save to excel and download the file
df.to_excel('spotify_ID2021.xlsx')
files.download('spotify_ID2021.xlsx')